In [2]:
import nltk
import json
import pandas as pd
import re
import gensim
from glob import glob
from gensim import corpora
from nltk.tokenize import word_tokenize
import csv
from collections import defaultdict

/Users/yehoon/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
captions = list()

In [5]:
with open('./total_datas.csv', encoding='UTF8') as f:
    rdr = csv.reader(f)
    for line in rdr:
        if line[1]!='text':
            captions.append({'date':line[0],'caption':line[1]})
# with open("../data/museum/caption2.txt", encoding='UTF8') as f:
#     captions=f.readlines()

In [12]:
# nltk.download('punkt')
ko_stop_words = """일상 시립 미술관 갤러리 전시회 북서울 서울 서울시 일 월 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 """

caption = pd.DataFrame(captions)
caption_concatenated = caption.groupby('date')['caption'].apply(''.join).reset_index()
# caption['clean_caption'] = caption['caption'].str.replace("^[가-힣]", " ")

In [13]:
caption_concatenated.head()

,date,caption
0,2017-10-27,"서울시립 북서울미술관, 어린이 전시'벌집도시와 소리숨바꼭질!' 개최[문화뉴스 MHN..."
1,2017-10-31,서울시가 추천하는 11월 문화예술 행사 150선【서울=뉴시스】박대로 기자 = 서울시...
2,2017-11-06,"소리로 느끼는 어린이 전시, ‘벌집도시와 소리숨바꼭질!’시각과 청각이 어우러진 미술..."
3,2017-12-01,"“오늘은 내가 탐험대장으로 변신해요”서울시립 북서울미술관, 어린이 상시 체험 프로그..."
4,2017-12-04,"[문화's 픽업] SeMA X 어린이의 창의력과 오감을 자극하는 체험 프로그램, '..."


In [14]:
from konlpy.tag import Mecab

In [15]:
mecab = Mecab()
caption_concatenated['clean_caption'] = caption_concatenated['caption'].apply(lambda x: [i for i in mecab.nouns(x) if i not in ko_stop_words])

In [17]:
caption_concatenated.head()

,date,caption,clean_caption
0,2017-10-27,"서울시립 북서울미술관, 어린이 전시'벌집도시와 소리숨바꼭질!' 개최[문화뉴스 MHN...","[어린이, 벌집, 도시, 소리, 숨바꼭질, 개최, 문화, 뉴스, 김민경, 기자, 벌..."
1,2017-10-31,서울시가 추천하는 11월 문화예술 행사 150선【서울=뉴시스】박대로 기자 = 서울시...,"[추천, 문화예술, 행사, 뉴시스, 박대, 기자, 문화, 예술, 프로그램, 시내, ..."
2,2017-11-06,"소리로 느끼는 어린이 전시, ‘벌집도시와 소리숨바꼭질!’시각과 청각이 어우러진 미술...","[소리, 어린이, 벌집, 도시, 소리, 숨바꼭질, 청각, 작품, 감상, 벌집, 도시..."
3,2017-12-01,"“오늘은 내가 탐험대장으로 변신해요”서울시립 북서울미술관, 어린이 상시 체험 프로그...","[오늘, 탐험대, 변신, 어린이, 상시, 체험, 프로그램, 배낭, 탐험가, 시범, ..."
4,2017-12-04,"[문화's 픽업] SeMA X 어린이의 창의력과 오감을 자극하는 체험 프로그램, '...","[문화, 픽업, 어린이, 창의력, 오감, 자극, 체험, 프로그램, 배낭, 탐험가, ..."


In [18]:
# tokenized_caption = caption['clean_caption'].apply(lambda x: x.split()) # 토큰화
# tokenized_caption = tokenized_caption.apply(lambda x: [item for item in x if item not in stop_words])

dictionary = corpora.Dictionary(caption_concatenated['clean_caption'])
corpus = [dictionary.doc2bow(text) for text in caption_concatenated['clean_caption']]

In [9]:
# # NUM_TOPICS = 10 #20개의 토픽, k=20
# for NUM_TOPIC in range(3, 21):
#     for NUM_WORD in range(3, 16):
#         print(f"NUM TOPIC: {NUM_TOPIC} / NUM_WORD: {NUM_WORD}")
#         ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPIC, id2word=dictionary, passes=30)
#         topics = ldamodel.print_topics(num_words=NUM_WORD)
#         for topic in topics:
#             print(topic)
#         print("================================\n")

In [24]:
# NUM_TOPICS = 10 #20개의 토픽, k=20
NUM_TOPIC = 5
NUM_WORD = 9
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPIC, id2word=dictionary, passes=30)
topics = lda_model.print_topics(num_words=NUM_WORD)
for topic in topics:
    print(topic)

(0, '0.018*"도시" + 0.015*"재생" + 0.012*"사업" + 0.011*"지역" + 0.010*"주민" + 0.007*"봉제" + 0.007*"사진" + 0.005*"공간" + 0.005*"월드"')
(1, '0.033*"데이비드호크니" + 0.021*"호크니" + 0.016*"작품" + 0.012*"그림" + 0.008*"미래" + 0.008*"작가" + 0.007*"그램" + 0.007*"데이비드" + 0.006*"문화"')
(2, '0.014*"비엔날레" + 0.011*"미디어" + 0.010*"작가" + 0.010*"작품" + 0.009*"시티" + 0.008*"공간" + 0.008*"예술" + 0.008*"사진" + 0.007*"삶"')
(3, '0.015*"작가" + 0.014*"작품" + 0.009*"한국" + 0.008*"디자인" + 0.008*"문화" + 0.006*"현대" + 0.006*"가나아트" + 0.006*"공예" + 0.006*"진행"')
(4, '0.022*"나무" + 0.017*"벽" + 0.015*"남자" + 0.014*"창문" + 0.013*"건물" + 0.012*"색" + 0.011*"작품" + 0.011*"탁자" + 0.010*"흰색"')


In [11]:
# test_doc = corpus[-1]
# vector = lda_model[test_doc]
# print(vector)

In [12]:
# lda_model.save('./konlpy_model.model')

In [19]:
# new_model = gensim.models.ldamodel.LdaModel.load("./konlpy_model.model")

In [20]:
result = new_model[corpus]
result

In [21]:
d = defaultdict(lambda: defaultdict(int))

for idx, rows in enumerate(result):
    for j,v in rows:
        d[j][idx] += v

tuple_to_pandas=pd.DataFrame(d).fillna(0)

result = pd.concat([caption_concatenated['date'],tuple_to_pandas],axis=1)       

IndexError: index 25817 is out of bounds for axis 1 with size 25817

In [14]:
test_doc = corpus[-1]
vector = new_model[test_doc]
print(vector)

[(0, 0.040000867), (1, 0.040000867), (2, 0.040000875), (3, 0.8399965), (4, 0.040000867)]


In [16]:
topics = new_model.print_topics(num_words=9)
for topic in topics:
    print(topic)

(0, '0.013*"도시" + 0.010*"백남준" + 0.010*"재생" + 0.009*"문화" + 0.008*"지역" + 0.008*"사업" + 0.007*"빌리지" + 0.007*"주민" + 0.006*"박물관"')
(1, '0.030*"작품" + 0.022*"작가" + 0.015*"시대" + 0.011*"미래" + 0.010*"한묵" + 0.010*"유감" + 0.009*"예술" + 0.008*"뉴욕" + 0.008*"사진"')
(2, '0.033*"그램" + 0.027*"스타" + 0.023*"데이비드호크니" + 0.015*"공간" + 0.015*"호크니" + 0.014*"맘" + 0.013*"주말" + 0.011*"데이트" + 0.011*"그림"')
(3, '0.049*"나무" + 0.043*"벽" + 0.041*"남자" + 0.031*"건물" + 0.030*"창문" + 0.030*"색" + 0.028*"흰색" + 0.026*"탁자" + 0.024*"의자"')
(4, '0.050*"공간" + 0.035*"어린이" + 0.016*"김주현" + 0.015*"작가" + 0.013*"이형구" + 0.012*"오민" + 0.012*"타이틀" + 0.011*"매치" + 0.010*"사물"')
